In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.12.0


In [7]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[ :1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [12]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [13]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
29/32 [==========================>...] - ETA: 0s - loss: 1.1607 - sparse_categorical_accuracy: 0.6853
Epoch 1: saving model to training_1\cp.ckpt
32/32 [==============================] - 2s 23ms/step - loss: 1.1215 - sparse_categorical_accuracy: 0.6980 - val_loss: 0.7556 - val_sparse_categorical_accuracy: 0.7680
Epoch 2/10
32/32 [==============================] - ETA: 0s - loss: 0.4274 - sparse_categorical_accuracy: 0.8770
Epoch 2: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.4274 - sparse_categorical_accuracy: 0.8770 - val_loss: 0.5582 - val_sparse_categorical_accuracy: 0.8260
Epoch 3/10
25/32 [======================>.......] - ETA: 0s - loss: 0.2756 - sparse_categorical_accuracy: 0.9250
Epoch 3: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 0.2942 - sparse_categorical_accuracy: 0.9180 - val_loss: 0.4928 - val_sparse_categorical_accuracy: 0.8360
Epoch 4/10
25/32 [=======

In [14]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [15]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3976 - sparse_categorical_accuracy: 0.1040 - 152ms/epoch - 5ms/step
Untrained model, accuracy: 10.40%


In [16]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4173 - sparse_categorical_accuracy: 0.8670 - 72ms/epoch - 2ms/step
Restored model, accuracy: 86.70%
